In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
headers = {'User-Agent': 'Text mining'}

# Construct the results table
boardgameList = ['stay cool']
listhashed = [i.replace(' ', '%20') for i in boardgameList]
summaryTable = pd.DataFrame()

for j in listhashed:
    # Load the webpage
    
    r = requests.get('https://boardgamegeek.com/geeksearch.php?action=search&objecttype=boardgame&q={}'.format(j), headers=headers)
    
    # Convert to BS
    bs = BeautifulSoup(r.text)
    
    # Find the right table
    table = bs.findAll(lambda tag: tag.name=='table' and tag.has_attr('cellpadding'))
    tableHTML = table[0]
    #table = bs.findAll("td", {"class":"collection_shop"})
    #tableHTML = table[0]
    
    # Convert the HTML of the table to pandas (very usefull function)
    pandasTable = pd.read_html(str(tableHTML), header=0)[0]
    
    # Extract all the links
    urls = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tableHTML.find_all('a')]
    urls = pd.DataFrame(urls)
    urls = urls[0][6]
#    urls = urls[0:-23]
#     urlsList.append(urls)
    
    # Merge the links into the DF
    pandasTable['link'] = urls
    
    # Drop unused columns
    pandasTable = pandasTable.drop(['Unnamed: 1', 'Shop'], axis = 1)
    
    # Create or extend the existing summary table
    if summaryTable.empty:
        summaryTable = pandasTable
    else:
        summaryTable = pd.concat([summaryTable, pandasTable],ignore_index=True)

In [46]:
display(summaryTable)

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters,link
0,6555,Stay Cool (2019),5.604,7.12,99,/boardgame/267475/stay-cool


In [47]:
# Loading the firefox webdriver
driver = webdriver.Chrome('/Users/korneliusz.krysiak/Dropbox/TextMining/chromedriver')

In [48]:
# Construct the results table
detailedTable = pd.DataFrame(columns=['Title','Average Rating','Number of ratings','Std.dev','Complexity','Number of Comments','Fans','Page Views','Overall Ranking','All Time Plays','Played This Month', 'Own', 'Previously Owned','For Trade','Want in Trade','Wishlist','Has Parts','Want Parts'])
detailedTable2 = pd.DataFrame()

for j in range(5):
    # Load the webpage
    driver.get('https://boardgamegeek.com{}/stats'.format(summaryTable['link'][j]))
    
    # Get the source code and make it BS
    pageSource = driver.page_source
    bs = BeautifulSoup(pageSource, "lxml")
    
    # Find the relevant table
    data = bs.findAll('div',{'class':'outline-item-description'})
    
    # Loop over and strip spaces
    data2 = []
    for i in data:
        data2.append(i.get_text().strip())

    # Remove the ',' 
    for i in range(len(data2)):
        data2[i] = data2[i].replace(",","")

    # Wrangle the columns
    data2[3] = data2[3][0:4]
    data2[0] = float(data2[0])
    data2[1] = int(data2[1]) 
    data2[2] = float(data2[2])
    data2[3] = float(data2[3]) 

    # Convert to int
    for i in range(len(data2[4:])+1):
        i = i+3
        data2[i] = int(data2[i])

    # Special case - two categories. We want to drop both
    if len(data2) == 19:
        data2.pop(8)
        data2.pop(8)
    else:
        data2.pop(8)
    
    # Insert the data
    data2.insert(0,summaryTable['Title'][j]) 
    detailedTable.loc[len(detailedTable)] = data2

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=79.0.3945.79)


In [ ]:
print(detailedTable)